In [1]:
import gmaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Google developer API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [2]:
#introducing Vehicle data
vehicle_df = pd.read_csv("vehicle_data.csv.gz", compression="gzip")

#take care of NaN velue
vehicle_df = vehicle_df.dropna()

#creating dataframe for car's for selective years
vehicle_year_df = vehicle_df.loc[vehicle_df['model_year'].astype(int) <= 2009]
#vehicle_year_df.head()

In [3]:
#calculating vehicle age till 2009 by model year 
veh_age = 2009 - vehicle_year_df['model_year'].astype(int)

#creating new data frame with new column for vehicle_age
new_vehicle_df = pd.DataFrame(vehicle_year_df)
new_vehicle_df['vehicle_age'] = veh_age
#new_vehicle_df.head()

In [4]:
#introducing zip code coordinates
zip_df = pd.read_csv('../Se7en/us-zip-code-latitude-and-longitude.csv')
zip_df = zip_df.rename (columns = {"Zip" : "zip"})

#take care of NaN velue
zip_df = zip_df.dropna()
#zip_df.head()

In [5]:
# find mean of vehicle age by zip code
veh_age_mean_df = new_vehicle_df.groupby('zip').agg({'vehicle_age':'mean'})
veh_age_mean_df = veh_age_mean_df.reset_index()
veh_age_mean_df.head()


,zip,vehicle_age
0,0,5.00
1,646,33.75
2,727,3.00
3,802,4.00
4,1002,0.50


In [6]:
#add zip code coordinates to vehicle age mean dataframe
vehicle_age_2009_df = pd.merge(veh_age_mean_df, zip_df, on=['zip','zip'], how="left")

#take care of NaN velue
vehicle_age_2009_df = vehicle_age_2009_df.dropna()
vehicle_age_2009_df.head()

,zip,vehicle_age,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
648,10001,5.845588,New York,NY,40.750742,-73.99653,-5.0,1.0
649,10002,5.068544,New York,NY,40.717040,-73.98700,-5.0,1.0
650,10003,7.640783,New York,NY,40.732509,-73.98935,-5.0,1.0
652,10005,6.469799,New York,NY,40.706019,-74.00858,-5.0,1.0
653,10006,5.413333,New York,NY,40.707904,-74.01342,-5.0,1.0


In [7]:
# introducing AQI data
AQI_df = pd.read_csv('../Se7en/AQIndex.csv')
AQI_df = AQI_df.dropna()
aqi_group = AQI_df.groupby ("Year")
aqi_2009_df = aqi_group.get_group (2009)
aqi_2009_df.head()

,Unnamed: 0,zip,AQI,Category Number,Category Name,Pollutant,Year
0,0,12032,90,2,Moderate,OZONE,2009
1,1,12108,90,2,Moderate,OZONE,2009
2,2,12134,90,2,Moderate,OZONE,2009
3,3,12139,90,2,Moderate,OZONE,2009
4,4,12164,90,2,Moderate,OZONE,2009


In [8]:
#merging AQI, Vehicle and coordinates data
vehicle_age_mean_2009_df = pd.merge(vehicle_age_2009_df, aqi_2009_df, on=['zip','zip'], how="left")

#take care of NaN velue
vehicle_age_mean_2009_df = vehicle_age_mean_2009_df.dropna()
vehicle_age_mean_2009_df

,zip,vehicle_age,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,Unnamed: 0,AQI,Category Number,Category Name,Pollutant,Year
0,10001,5.845588,New York,NY,40.750742,-73.99653,-5.0,1.0,991.0,44.0,1.0,Good,OZONE,2009.0
1,10002,5.068544,New York,NY,40.717040,-73.98700,-5.0,1.0,992.0,44.0,1.0,Good,OZONE,2009.0
2,10003,7.640783,New York,NY,40.732509,-73.98935,-5.0,1.0,993.0,44.0,1.0,Good,OZONE,2009.0
3,10005,6.469799,New York,NY,40.706019,-74.00858,-5.0,1.0,995.0,44.0,1.0,Good,OZONE,2009.0
4,10006,5.413333,New York,NY,40.707904,-74.01342,-5.0,1.0,996.0,44.0,1.0,Good,OZONE,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,14898,5.763158,Woodhull,NY,42.065376,-77.42392,-5.0,1.0,235.0,80.0,2.0,Moderate,OZONE,2009.0
2045,14901,5.823712,Elmira,NY,42.091924,-76.79217,-5.0,1.0,236.0,80.0,2.0,Moderate,OZONE,2009.0
2047,14903,6.749054,Elmira,NY,42.129274,-76.84497,-5.0,1.0,237.0,80.0,2.0,Moderate,OZONE,2009.0
2048,14904,5.899058,Elmira,NY,42.072024,-76.80393,-5.0,1.0,238.0,80.0,2.0,Moderate,OZONE,2009.0


In [9]:
#Plot Heatmap for vehicle age by zip code
locations_09 = vehicle_age_mean_2009_df[["Latitude", "Longitude"]].astype(float)
AQI_09 = vehicle_age_mean_2009_df["AQI"].astype(float)


#create heat layer for vehicle age by zip
heat_layer_09 = gmaps.heatmap_layer(locations_09, weights=AQI_09, 
                                  dissipating=False, max_intensity=500,                                  
                                  point_radius=0.25)

#convert vehicle age mean to list
vehicle_age_mean_09 = vehicle_age_mean_2009_df["vehicle_age"].tolist()
#vehicle_age_mean_09

vehicle_age_layer_09 = gmaps.symbol_layer(locations_09, fill_color='rgba(0, 150, 0, 0.4)',
    fill_opacity=0.1,                                       
    stroke_color='rgba(0, 0, 150, 0.4)', scale=[int(vehicle_age) for vehicle_age in vehicle_age_mean_09],
    info_box_content=[f"vehicle age mean is: {vehicle_age}" for vehicle_age in vehicle_age_mean_09])

 

#combine both maps
fig = gmaps.figure()
fig.add_layer(heat_layer_09)
fig.add_layer(vehicle_age_layer_09)
fig

Figure(layout=FigureLayout(height='420px'))